# VGG-11层

- padding  same

- img  224 x 224
- conv_1 f_64 k_3x3  s_1x1    => 224x224x64
- pool_1  k_2x2  s_2x2          => 112x112x64

- conv_2 f_128 k_3x3  s_1x1    => 112x112x128
- pool_2  k_2x2  s_2x2           => 56x56x128

- conv_3 f_256 k_3x3  s_1x1    => 56x56x256
- conv_4 f_256 k_3x3  s_1x1    => 56x56x256
- pool_3  k_2x2  s_2x2           => 28x28x256

- conv_5 f_512 k_3x3  s_1x1    => 28x38x512
- conv_6 f_512 k_3x3  s_1x1    => 38x38x512
- pool_3  k_2x2  s_2x2           => 14x14x512

- conv_7 f_512 k_3x3  s_1x1    => 14x14x512
- conv_8 f_512 k_3x3  s_1x1    => 14x14x512
- pool_3  k_2x2  s_2x2           => 7x7x512

- dense_9  in_25088 out_4096 => 4096
- dense_10 in_4096 out_4096 => 4096
- dense_11 in_4096 out_1000 => 1000

In [1]:
import pandas as pd

In [2]:
picture224x224_base_path="/Users/zhouwencheng/Desktop/Grass/data/picture/picture224x224"
path_train_base = picture224x224_base_path + "/train/"
path_test_base = picture224x224_base_path + "/test/"

df_train = pd.read_csv(path_train_base + 'list.csv', index_col=0)
df_test = pd.read_csv(path_test_base + 'list.csv', index_col=0)
len(df_train)

5436

In [3]:
df_train.head()

,name,label
0,217_10.png,217
1,179_17.png,179
2,204_4.png,204
3,118_1.png,118
4,079_5.png,79


In [4]:
df_test.head()

,name,label
0,315_9.png,315
1,295_13.png,295
2,222_7.png,222
3,345_18.png,345
4,077_12.png,77


In [5]:
# readImg
import matplotlib.pyplot as plt
import numpy as np

x_train = []
for name in df_train.name:
    path = path_train_base + name
    img = plt.imread(path).ravel()
    x_train.append(img)
x_train = np.array(x_train)

In [6]:
# readImg
import matplotlib.pyplot as plt
import numpy as np

x_test = []
for name in df_test.name:
    path = path_test_base + name
    img = plt.imread(path).ravel()
    x_test.append(img)
x_test = np.array(x_test)

In [7]:
y_train = df_train.label.values
y_test = df_test.label.values

print(x_train.shape)
print(x_test.shape)
print(y_train.shape)
print(y_test.shape)

(5436, 50176)
(1651, 50176)
(5436,)
(1651,)


In [8]:
from sklearn.preprocessing import OneHotEncoder 
enc = OneHotEncoder(sparse=False)
enc.fit(y_train.reshape([-1, 1]))
y_train_one = enc.transform(y_train.reshape([-1, 1]))
y_test_one = enc.transform(y_test.reshape([-1, 1])) 

/Users/zhouwencheng/Desktop/Grass/02Study/02PythonEnv/1env2/lib/python3.7/site-packages/sklearn/preprocessing/_encoders.py:371: FutureWarning: The handling of integer data will change in version 0.22. Currently, the categories are determined based on the range [0, max(values)], while in the future they will be determined based on the unique values.
If you want the future behaviour and silence this warning, you can specify "categories='auto'".
In case you used a LabelEncoder before this OneHotEncoder to convert the categories to integers, then you can now use the OneHotEncoder directly.
  warnings.warn(msg, FutureWarning)


In [9]:
def data_batch(x_data, n_batch, batch_size):
    len_train = len(x_data)
    total_batch = len_train // batch_size
    if total_batch%batch_size > 0:
        total_batch += 1 
    get_batch = n_batch%total_batch
    first_index = get_batch * batch_size
    last_index = get_batch*batch_size + batch_size
    if last_index > len_train:
        last_index = len_train
    return x_data[first_index: last_index]  

In [10]:
import tensorflow as tf

In [11]:
img_type = 386

In [12]:
x = tf.placeholder(dtype=tf.float32, shape=[None, 224*224])
y = tf.placeholder(dtype=tf.float32, shape=[None, img_type])

x_flat = tf.reshape(x, shape=[-1, 224, 224, 1]) 

In [13]:
conv_1 = tf.layers.conv2d(inputs=x_flat, filters=64, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
pool_1 = tf.layers.max_pooling2d(inputs=conv_1, pool_size=(2, 2), strides=(2, 2), padding='same')

conv_2 = tf.layers.conv2d(inputs=pool_1, filters=128, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
pool_2 = tf.layers.max_pooling2d(inputs=conv_2, pool_size=(2, 2), strides=(2, 2), padding='same')

conv_3 = tf.layers.conv2d(inputs=pool_2, filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
conv_4 = tf.layers.conv2d(inputs=conv_3, filters=256, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
pool_3 = tf.layers.max_pooling2d(inputs=conv_4, pool_size=(2, 2), strides=(2, 2), padding='same')

conv_5 = tf.layers.conv2d(inputs=pool_3, filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
conv_6 = tf.layers.conv2d(inputs=conv_5, filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
pool_4 = tf.layers.max_pooling2d(inputs=conv_6, pool_size=(2, 2), strides=(2, 2), padding='same')

conv_7 = tf.layers.conv2d(inputs=pool_4, filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
conv_8 = tf.layers.conv2d(inputs=conv_7, filters=512, kernel_size=(3, 3), strides=(1, 1), padding='same',activation=tf.nn.relu)
pool_5 = tf.layers.max_pooling2d(inputs=conv_8, pool_size=(2, 2), strides=(2, 2), padding='same')

pool_5_flat = tf.reshape(pool_5, shape=[-1, 7*7*512])

dense_1 = tf.layers.dense(pool_5_flat, 1024, activation=tf.nn.relu)
dense_1 = tf.nn.dropout(dense_1, keep_prob=0.5)
dense_2 = tf.layers.dense(dense_1, 1024, activation=tf.nn.relu)
dense_2 = tf.nn.dropout(dense_2, keep_prob=0.5)
prediction = tf.layers.dense(dense_2, img_type , activation=tf.nn.softmax)

Instructions for updating:
Use keras.layers.conv2d instead.
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Use keras.layers.max_pooling2d instead.
Instructions for updating:
Use keras.layers.dense instead.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.


In [14]:
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=prediction))
train = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

correct_prediction = tf.equal(x=tf.math.argmax(y, 1), y=tf.math.argmax(prediction, 1))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, dtype = tf.float32))

Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.



In [15]:
x_batch = data_batch(x_train, n_batch=0, batch_size=100)
y_batch = data_batch(y_train_one, n_batch=0, batch_size=100)
print("x_batch:", x_batch.shape)
print("y_batch:", y_batch.shape)

x_batch: (100, 50176)
y_batch: (100, 386)


In [ ]:
print("BG") 

to_saver = tf.train.Saver()  

with tf.Session() as sess:  
#     sess.run(tf.global_variables_initializer())
    saver = tf.train.import_meta_graph('save/660/LeNet.ckpt.meta')
    if saver:
        saver.restore(sess, 'save/660/LeNet.ckpt') 
    
    for step in range(100): 
        for batch in range(100):
            x_batch = data_batch(x_train, n_batch=batch, batch_size=100)
            y_batch = data_batch(y_train_one, n_batch=batch, batch_size=100)
            sess.run(train, feed_dict = {x:x_batch, y:y_batch})
            
#             print("x_batch:", x_batch.shape)
#             print("y_batch:", y_batch.shape)
             
            x_test_batch = data_batch(x_test, n_batch=0, batch_size=100)
            y_test_batch = data_batch(y_test_one, n_batch=0, batch_size=100)
            
            this_loss, acc = sess.run([loss, accuracy], feed_dict = {x:x_test_batch, y:y_test_batch})
            print('step:', step, 'loss:', this_loss, 'accuracy:', acc)  
            to_saver.save(sess, "save/660/LeNet.ckpt")
        
    print("OK")

BG
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from save/660/LeNet.ckpt
step: 0 loss: 5.9502788 accuracy: 0.01
step: 0 loss: 5.96028 accuracy: 0.0
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.96028 accuracy: 0.0
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
step: 0 loss: 5.95028 accuracy: 0.01
